In [3]:
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


KeyboardInterrupt: 

In [ ]:
# disorder_name = input("Choose one of anxiety, bipolar-disorder ,clinical-depression, post-traumatic-stress-disorder and obsessive-compulsive-disorder")

disorder_name='anxiety'

In [ ]:
# user_rant=input("Tell us how you feel!")

user_rant= 'i feel anxious'

In [ ]:
df=pd.read_excel('mental_health_data.xlsx')
df.head()

NameError: name 'pd' is not defined

In [ ]:
def remove_stopwords(text):
    words = nltk.word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_words = [word for word in filtered_words if word.isalnum()]
    return " ".join(filtered_words)

# Assuming df['Concern'] contains your text data
df['Concern'] = df['Concern'].str.lower()
# df['Concern'] = df['Concern'].astype(str).apply(remove_stopwords)
print(df['Concern'])

0        hello.  i am a 51 y/o man, suffering from bipo...
1        hello.  i am a 51 y/o man, suffering from bipo...
2        trigger warning‚ä¶,having a breakdown of crisi...
3        trigger warning‚ä¶,having a breakdown of crisi...
4        trigger warning‚ä¶,having a breakdown of crisi...
                               ...                        
29049    i haven't been active on here in awhile...ice ...
29050    i haven't been active on here in awhile...ice ...
29051    i haven't been active on here in awhile...ice ...
29052    i haven't been active on here in awhile...ice ...
29053    i haven't been active on here in awhile...ice ...
Name: Concern, Length: 29054, dtype: object


In [ ]:
df_disorder=df[df["tag"]==disorder_name]

In [ ]:
df_disorder= df_disorder[['Concern']].drop_duplicates().reset_index(drop=True)
df_disorder['User Concern']=user_rant

In [ ]:
df_disorder

,Concern,User Concern
0,beginning to use some not good coping skills. ...,i feel anxious
1,hi everyone. i just joined this forum due to w...,i feel anxious
2,"the description below is me.""exercise and str...",i feel anxious
3,does anyone feel like you concentrate on one t...,i feel anxious
4,i just called my grandparents to tell them im ...,i feel anxious
...,...,...
1394,has anyone here had prolia ( denosumab ) as th...,i feel anxious
1395,"""anxiety, being what anxiety is, worked its wa...",i feel anxious
1396,i've seen a similar post on here but it's not ...,i feel anxious
1397,so i've experienced general health anxiety for...,i feel anxious


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Calculate the sentiment score for the single "User Concern" value
user_concern_sentiment = analyzer.polarity_scores(df_disorder.iloc[0]['User Concern'])['compound']

# Apply the sentiment score to all rows in the "Concern" column

df_disorder['Concern_Sentiment'] = df_disorder['Concern'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
df_disorder['User_Concern_Sentiment'] = user_concern_sentiment

# Print the updated DataFrame
df_disorder

,Concern,User Concern,Concern_Sentiment,User_Concern_Sentiment
0,beginning to use some not good coping skills. ...,i feel anxious,0.6043,-0.25
1,hi everyone. i just joined this forum due to w...,i feel anxious,-0.3762,-0.25
2,"the description below is me.""exercise and str...",i feel anxious,-0.5212,-0.25
3,does anyone feel like you concentrate on one t...,i feel anxious,-0.6980,-0.25
4,i just called my grandparents to tell them im ...,i feel anxious,-0.9590,-0.25
...,...,...,...,...
1394,has anyone here had prolia ( denosumab ) as th...,i feel anxious,0.6335,-0.25
1395,"""anxiety, being what anxiety is, worked its wa...",i feel anxious,-0.9590,-0.25
1396,i've seen a similar post on here but it's not ...,i feel anxious,0.9425,-0.25
1397,so i've experienced general health anxiety for...,i feel anxious,-0.9874,-0.25


In [ ]:
# We assume that a sentiment score of greater than 0.05 (based on Vader) of the user rant means the user is talking about something cheerful
# Same with all concerns in our data

# In other words, we need to filter our for negative user rants and negative concerns in our data

df_disorder= df_disorder[(df_disorder['Concern_Sentiment']<0.05) & (df_disorder['User_Concern_Sentiment']<0.05)]

In [ ]:
# df_disorder=df_disorder[:100]

In [ ]:
nlp = spacy.load('en_core_web_md')
def word_vector_similarity(text1, text2):
    doc1 = nlp(text1)
    doc2 = nlp(text2)
    return doc1.similarity(doc2)

# Create a new DataFrame with product name, product review, and similarity score
# similarity_score = pd.DataFrame({
#     'Product Name': df['item'],
#     'Concern': df_disorder['Concern'],
#     'Similarity Score': ''
# })

# this adds the simlarity score of each comment in the concern column with the user input
df_disorder['Similarity Score'] = df_disorder['Concern'].apply(word_vector_similarity, text2=user_rant)

C:\Users\rahul\AppData\Local\Temp\ipykernel_40176\3849683405.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_disorder['Similarity Score'] = df_disorder['Concern'].apply(word_vector_similarity, text2=user_rant)


In [ ]:
df_disorder=df_disorder[df_disorder['Similarity Score']>0.5].sort_values(by='Similarity Score', ascending=False).reset_index(drop=True)[:10]

In [ ]:
df_sug= df_disorder.merge(df, on='Concern', how='left')[['Concern','User Concern','Similarity Score','Concern_Sentiment','User_Concern_Sentiment','Suggestion']]

In [ ]:
# from gensim.models import Phrases
# from gensim.models.phrases import Phraser
# from gensim.models import LdaModel
# from gensim import corpora

# df_sug['Suggestion']= df_sug['Suggestion'].astype(str)
# df_sug['Suggestion']= df_sug['Suggestion'].apply(remove_stopwords)

# # Create bigrams or multi-word phrases
# bigram = Phrases(df_sug['Suggestion'], min_count=5, threshold=100)
# bigram_phraser = Phraser(bigram)

# # Preprocess the text and apply the bigram phraser
# df_sug['Suggestion'] = df_sug['Suggestion'].str.lower()
# df_sug['Suggestion'] = df_sug['Suggestion'].apply(word_tokenize)
# df_sug['Suggestion'] = df_sug['Suggestion'].apply(lambda tokens: bigram_phraser[tokens])



In [ ]:
# df_sug['Suggestion']

In [ ]:
# # Create a dictionary and a corpus for LDA
# dictionary = corpora.Dictionary(df_sug['Suggestion'])
# corpus = [dictionary.doc2bow(text) for text in df_sug['Suggestion']]

# # Train an LDA model
# num_topics = 2  # Set the number of topics you want
# lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# # Print the top phrases for each topic
# for topic_num in range(num_topics):
#     topic_words = lda_model.show_topic(topic_num)
#     print(f"Topic {topic_num + 1}:")
#     for word, score in topic_words:
#         print(f"{word}: {score}")
#     print()

In [ ]:
# from gensim.models import Phrases
# from gensim.models.phrases import Phraser
# from gensim.models import LdaModel
# from gensim import corpora
# from nltk.tokenize import word_tokenize

# df_sug= df_disorder.merge(df, on='Concern', how='left')[['Concern','User Concern','Similarity Score','Concern_Sentiment','User_Concern_Sentiment','Suggestion']]

# df_sug['Suggestion']= df_sug['Suggestion'].astype(str)
# df_sug['Suggestion']= df_sug['Suggestion'].apply(remove_stopwords)

# # Tokenize the sentences in the 'Suggestion' column
# df_sug['Suggestion'] = df_sug['Suggestion'].str.lower()
# df_sug['Suggestion'] = df_sug['Suggestion'].apply(word_tokenize)

# # Create bigrams or multi-word phrases
# bigram = Phrases(df_sug['Suggestion'], min_count=2, threshold=20)
# bigram_phraser = Phraser(bigram)

# # Apply the bigram phraser to the tokenized text
# df_sug['Suggestion'] = df_sug['Suggestion'].apply(lambda tokens: bigram_phraser[tokens])

# # Create a dictionary and a corpus for LDA
# dictionary = corpora.Dictionary(df_sug['Suggestion'])
# corpus = [dictionary.doc2bow(text) for text in df_sug['Suggestion']]

# # Train an LDA model
# num_topics = 2  # Set the number of topics you want
# lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# # Print the top phrases for each topic
# for topic_num in range(num_topics):
#     topic_words = lda_model.show_topic(topic_num)
#     print(f"Topic {topic_num + 1}:")
#     for word, score in topic_words:
#         print(f"{word}: {score}")
#     print()

In [ ]:
# nltk.download('averaged_perceptron_tagger')

In [ ]:
# import pandas as pd
# import nltk
# from nltk import word_tokenize, pos_tag, RegexpParser

# df_sug= df_disorder.merge(df, on='Concern', how='left')[['Concern','User Concern','Similarity Score','Concern_Sentiment','User_Concern_Sentiment','Suggestion']]

# df_sug['Suggestion']= df_sug['Suggestion'].astype(str)
# # df_sug['Suggestion']= df_sug['Suggestion'].apply(remove_stopwords)
# # Define a function to extract noun phrases
# def extract_noun_phrases(text):
#     words = word_tokenize(text)
#     tagged = pos_tag(words)
#     grammar = r"""
#         NP: {<DT>?<JJ>*<NN.*>+}
#     """
#     cp = RegexpParser(grammar)
#     tree = cp.parse(tagged)
#     noun_phrases = []
#     for subtree in tree.subtrees(filter=lambda t: t.label() == 'NP'):
#         noun_phrase = " ".join(word for word, tag in subtree.leaves())
#         noun_phrases.append(noun_phrase)
#     return noun_phrases

# # Apply the function to each suggestion
# df_sug['Noun_Phrases'] = df_sug['Suggestion'].apply(extract_noun_phrases)

# # Print the extracted noun phrases
# print(df_sug['Noun_Phrases'])

In [ ]:
# df_sug= df_disorder.merge(df, on='Concern', how='left')[['Concern','User Concern','Similarity Score','Concern_Sentiment','User_Concern_Sentiment','Suggestion']]

# df_sug['Suggestion']= df_sug['Suggestion'].astype(str)

# # Load a pre-trained spaCy model (English)
# nlp = spacy.load("en_core_web_md")

# # Extract unique verbs from all suggestions
# unique_verbs = set()  # Initialize an empty set to store unique verbs

# for suggestion in df_sug['Suggestion']:
#     # Process the text with spaCy
#     doc = nlp(suggestion)
    
#     # Extract verbs (tokens with the POS tag "VERB") and add them to the set
#     extracted_verbs = {token.text for token in doc if token.pos_ == "VERB"}
    
#     # Update the set of unique verbs with the extracted verbs
#     unique_verbs.update(extracted_verbs)

# # Convert the set of unique verbs back to a list (if needed)
# unique_verbs_list = list(unique_verbs)

# # Print the unique set of verbs
# print(unique_verbs_list)

In [ ]:
# import pandas as pd
# import spacy
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import NMF



# # Load a pre-trained spaCy model (English)
# nlp = spacy.load("en_core_web_md")

# # Define a function to filter verbs
# def filter_verbs(text):
#     doc = nlp(text)
#     verb_tokens = [token.text for token in doc if token.pos_ == "VERB"]
#     return " ".join(verb_tokens)

# # Apply the filter to the DataFrame
# df_sug['Filtered_Text'] = df_sug['Suggestion'].apply(filter_verbs)

# # Create a TF-IDF vectorizer
# tfidf_vectorizer = TfidfVectorizer(max_features=10)  # Adjust the number of features/topics as needed

# # Fit and transform the filtered text data
# tfidf_matrix = tfidf_vectorizer.fit_transform(df_sug['Filtered_Text'])

# # Apply NMF for topic modeling
# num_topics = 2  # Set the number of topics you want
# nmf_model = NMF(n_components=num_topics)
# nmf_matrix = nmf_model.fit_transform(tfidf_matrix)

# # Get the top words for each topic
# feature_names = tfidf_vectorizer.get_feature_names_out()
# top_words_per_topic = []
# for topic_idx, topic in enumerate(nmf_model.components_):
#     top_words_idx = topic.argsort()[-5:][::-1]  # Adjust the number of top words per topic as needed
#     top_words = [feature_names[i] for i in top_words_idx]
#     top_words_per_topic.append(top_words)

# # Print the top words for each topic
# for topic_idx, top_words in enumerate(top_words_per_topic):
#     print(f"Topic {topic_idx + 1}: {', '.join(top_words)}")

In [ ]:
# import numpy as np

# df_sug= df_disorder.merge(df, on='Concern', how='left')[['Concern','User Concern','Similarity Score','Concern_Sentiment','User_Concern_Sentiment','Suggestion']]
# df_sug['Suggestion']= df_sug['Suggestion'].astype(str)

# # Combine rows into one paragraph
# paragraph = ' '.join(df_sug['Suggestion'])

# # Load a pre-trained spaCy model (English)
# nlp = spacy.load('en_core_web_md')
# nlp.max_length = 2000000 # or even higher

# # Split the paragraph into sentences
# doc = nlp(paragraph)
# sentences = [sent.text for sent in doc.sents]

# # Initialize a matrix to store sentence similarity scores
# num_sentences = len(sentences)
# similarity_matrix = np.zeros((num_sentences, num_sentences))

# # Calculate sentence similarity using word vector similarity
# for i in range(num_sentences):
#     for j in range(i, num_sentences):  # Only calculate each pair once
#         if i != j:
#             similarity = nlp(sentences[i]).similarity(nlp(sentences[j]))
#             similarity_matrix[i, j] = similarity
#             similarity_matrix[j, i] = similarity  # Symmetric

# # Print the sentence similarity matrix (optional)
# print("Sentence Similarity Matrix:")
# print(similarity_matrix)


In [ ]:
df_sug= df_disorder.merge(df, on='Concern', how='left')[['Concern','User Concern','Similarity Score','Concern_Sentiment','User_Concern_Sentiment','Suggestion']]
df_sug['Suggestion']= df_sug['Suggestion'].astype(str)

# Combine rows into one paragraph
paragraph = ' '.join(df_sug['Suggestion'])

# Load a pre-trained spaCy model (English)
nlp = spacy.load('en_core_web_md')
nlp.max_length = 2000000 # or even higher

# Split the paragraph into sentences
doc = nlp(paragraph)
sentences = [sent.text for sent in doc.sents]

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

df_sug = df_disorder.merge(df, on='Concern', how='left')[['Concern', 'User Concern', 'Similarity Score', 'Concern_Sentiment', 'User_Concern_Sentiment', 'Suggestion']]
df_sug['Suggestion'] = df_sug['Suggestion'].astype(str)

# Combine rows into one paragraph
paragraph = ' '.join(df_sug['Suggestion'])

# Load a pre-trained spaCy model (English)
nlp = spacy.load('en_core_web_md')
nlp.max_length = 2000000  # or even higher

# Split the paragraph into sentences
doc = nlp(paragraph)
sentences = [sent.text for sent in doc.sents]

# Initialize a matrix to store sentence similarity scores
num_sentences = len(sentences)
similarity_matrix = np.zeros((num_sentences, num_sentences))

# Calculate sentence similarity using cosine similarity
for i in range(num_sentences):
    for j in range(i, num_sentences):  # Only calculate each pair once
        if i != j:
            vector1 = nlp(sentences[i]).vector
            vector2 = nlp(sentences[j]).vector
            similarity = cosine_similarity([vector1], [vector2])[0][0]
            similarity_matrix[i, j] = similarity
            similarity_matrix[j, i] = similarity  # Symmetric

# Print the sentence similarity matrix (optional)
print("Sentence Similarity Matrix:")
print(similarity_matrix)

Sentence Similarity Matrix:
[[0.         0.67193466 0.73401952 ... 0.5498715  0.73435664 0.6273554 ]
 [0.67193466 0.         0.80114758 ... 0.81205541 0.70404387 0.79551411]
 [0.73401952 0.80114758 0.         ... 0.84287786 0.82239437 0.83103955]
 ...
 [0.5498715  0.81205541 0.84287786 ... 0.         0.76752681 0.89502686]
 [0.73435664 0.70404387 0.82239437 ... 0.76752681 0.         0.79425079]
 [0.6273554  0.79551411 0.83103955 ... 0.89502686 0.79425079 0.        ]]


In [ ]:
# Sum up the similarity values for each sentence
sentence_similarity_scores = np.sum(similarity_matrix, axis=1)

# Create a list of (sentence, score) tuples
sentence_scores_tuples = [(sentence, score) for sentence, score in zip(sentences, sentence_similarity_scores)]

# Sort the tuples by score in descending order
sentence_scores_tuples_sorted = sorted(sentence_scores_tuples, key=lambda x: x[1], reverse=True)

# Select the top 10 sentences with the highest scores
top_10_sentences = [sentence for sentence, score in sentence_scores_tuples_sorted[:10]]

# Print the top 10 sentences
for i, sentence in enumerate(top_10_sentences, 1):
    print(f"Top {i}: {sentence}")

Top 1: You need to understand yourself  and heal  and accept any past problems, thiscan of course  be a  problem and you need that support that seems to be lacking  at this time BOB Hi, can I just say I feel for you, my husband also suffers severe depression & I know  from experience what your saying.
Top 2: so I am limited Also' for me' People You have nothing to be sorry for, my dear, you must not take the blame for that which you did not do, though those who are to blame will readily let you burden yourself with their guilt.
Top 3: There is nothing as difficult as forgiving someone who keeps doing the same thing over and over when they know it is wrong, so I forgave and distanced myself, that way I gave myself time to heal bcz they were out of sight.
Top 4: You may try, I do this and it helps me a bit, to take a list of what is done to you and what you do to others.
Top 5: No one will ever understand what going through mental health issues are like, so why bother to dig the hole dee

In [ ]:
len(' '.join(top_10_sentences))

2025

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from collections import Counter

# Load the spaCy model
nlp = spacy.load('en_core_web_md')

# Your long text goes here
text = ' '.join(top_10_sentences)

# Tokenize the text and remove stop words
doc = nlp(text)
tokens = [token.text for token in doc if token.text not in STOP_WORDS]

# Calculate word frequency
word_freq = Counter(tokens)

# Calculate sentence scores based on word frequency
sentences = [sent.text for sent in doc.sents]
sentence_scores = {sent: sum([word_freq[word] for word in sent.split()]) for sent in sentences}

# Get the top N sentences for summary
summary_sentences = [sentence for sentence, score in sorted(sentence_scores.items(), key=lambda x: x[1], reverse=True)[:3]]

# Join the summary sentences to create the summary
summary = ' '.join(summary_sentences)

# Print the summary
print(summary)

You need to understand yourself  and heal  and accept any past problems, thiscan of course  be a  problem and you need that support that seems to be lacking  at this time BOB Hi, can I just say I feel for you, my husband also suffers severe depression & I know  from experience what your saying. hi there I think your fearing the future because with everything going on right now in the world many of us are at the minute but please get to your gp and tell them how it‚Äôs effecting your mental health we always fear what we can‚Äôt control don‚Äôt we it will pass with the right help  You can always ring the Samaritans they pretty good at talking to us when we anxious  Take care  Nat We don't always know how we seem to other people;  someone once said that I was "aloof" - "stand offish", when actually I was shy, so it's worth trying to find out what it is that people don't stay for, and are they really the people you would want to stay for long.   


In [ ]:
top_10_sentences

['You need to understand yourself  and heal  and accept any past problems, thiscan of course  be a  problem and you need that support that seems to be lacking  at this time BOB Hi, can I just say I feel for you, my husband also suffers severe depression & I know  from experience what your saying.',
 "so I am limited Also' for me' People You have nothing to be sorry for, my dear, you must not take the blame for that which you did not do, though those who are to blame will readily let you burden yourself with their guilt.",
 'There is nothing as difficult as forgiving someone who keeps doing the same thing over and over when they know it is wrong, so I forgave and distanced myself, that way I gave myself time to heal bcz they were out of sight.',
 'You may try, I do this and it helps me a bit, to take a list of what is done to you and what you do to others.',
 'No one will ever understand what going through mental health issues are like, so why bother to dig the hole deeper for yourself.

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import nltk

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Sample paragraph
paragraph= ' '.join(top_10_sentences)
# Split paragraph into sentences
sentences = nltk.sent_tokenize(paragraph)

# Define a threshold for action phrase detection
threshold = 0.5

# Process each sentence
for sentence in sentences:
    # Tokenize and convert to tensor
    input_ids = tokenizer.encode(sentence, add_special_tokens=True, max_length=64, truncation=True, padding=True, return_tensors="pt")
    
    # Make prediction
    with torch.no_grad():
        outputs = model(input_ids)
        action_probability = torch.softmax(outputs.logits, dim=1)[0][1].item()

    # Check if the sentence contains an action phrase
    is_action_phrase = action_probability > threshold

    if is_action_phrase:
        print("Action Phrase Detected:", sentence)
    else:
        print("Not an Action Phrase:", sentence)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Action Phrase Detected: You need to understand yourself  and heal  and accept any past problems, thiscan of course  be a  problem and you need that support that seems to be lacking  at this time BOB Hi, can I just say I feel for you, my husband also suffers severe depression & I know  from experience what your saying.
Action Phrase Detected: so I am limited Also' for me' People You have nothing to be sorry for, my dear, you must not take the blame for that which you did not do, though those who are to blame will readily let you burden yourself with their guilt.
Action Phrase Detected: There is nothing as difficult as forgiving someone who keeps doing the same thing over and over when they know it is wrong, so I forgave and distanced myself, that way I gave myself time to heal bcz they were out of sight.
Action Phrase Detected: You may try, I do this and it helps me a bit, to take a list of what is done to you and what you do to others.
Action Phrase Detected: No one will ever understan

In [ ]:
df[2:3]

NameError: name 'df' is not defined

In [ ]:
import spacy

# Load the spaCy NER model
nlp = spacy.load("en_core_web_md")

# Sample text
text = ' '.join(top_10_sentences)

# Process the text with spaCy
doc = nlp(text)

# Define a list of action-related entity labels
action_labels = ["VERB", "AUX", "ADV", "PART"]

# Initialize a list to store action phrases
action_phrases = []

# Initialize variables to track phrases
current_phrase = ""
phrase_tokens = []

# Iterate through tokens and extract action-related phrases
for token in doc:
    if token.pos_ in action_labels:
        # If the token is an action-related word, add it to the current phrase
        phrase_tokens.append(token.text)
    else:
        # If the token is not an action-related word, check if the current phrase is non-empty
        if phrase_tokens:
            # Join the tokens to form a phrase and add it to the list of action phrases
            action_phrases.append(" ".join(phrase_tokens))
            # Reset variables for the next phrase
            phrase_tokens = []

# Check if there's a remaining phrase at the end
if phrase_tokens:
    action_phrases.append(" ".join(phrase_tokens))

# Print extracted action phrases
print("Action Phrases:", action_phrases)

Action Phrases: ['need to understand', 'heal', 'accept', 'be', 'need', 'seems to be lacking', 'can', 'just say', 'feel', 'also suffers', 'know', 'so', 'am', 'Also', 'have', 'to be', 'must not take', 'did not do', 'are to blame will readily let', 'burden', 'is', 'as', 'forgiving', 'keeps doing', 'over', 'know', 'is', 'so', 'forgave', 'distanced', 'gave', 'to heal', 'were', 'may try', 'do', 'helps', 'to take', 'is done', 'do', 'will ever understand', 'going', 'are', 'so', 'bother to dig', 'deeper', 'there', 'think', 'fearing', 'going', 'right now', 'are', 'get', 'tell', 'it‚Äôs effecting', 'always fear', 'can‚Äôt', 'will pass', 'can always ring', 'pretty', 'talking', 'Take', 'think mostly', 'need to keep talking', 'no matter', 'sounds', 'Never', 'always feeling', 'are demanding', 'feeling', 'annoy', 'Stepping back to take', 'are dealing', 'acknowledges', 'is', 'shows', 'are not as', 'may feel', "do n't always know", 'seem', 'once said', 'was', 'actually', 'was', "'s", 'trying to find', '